In [21]:
import pickle
import pandas as pd

import folium
from folium import plugins
from folium.plugins import HeatMap

### Read measurements data from two consecutive days 

In [15]:
# sample data based on which the visualisations are created
day1 = pd.read_pickle(r'./measurements/oneday.pickle')
day2 = pd.read_pickle(r'./measurements/oneday2.pickle')

# 1 hour, in each hour data from sensors
# each tuple data: (pm1, pm10, pm25, longitude, latitude)
print(f"Count of sensors measuring air quality: {len(day1[0])}")

### Merge data from pickles to DataFrame

In [22]:
from datetime import datetime, timedelta

time = datetime(2022, 12, 20)
air_df = pd.DataFrame(columns=['PM1', 'PM10', 'PM25', 'latitude', 'longitude'])

for day in [day1, day2]:
    for hour_data in day:
        new_df = pd.DataFrame(hour_data, columns=['PM1', 'PM10', 'PM25', 'latitude', 'longitude'])
        new_df['timestamp'] = time
        time = time + timedelta(hours=1)
        air_df = pd.concat([air_df, new_df])

In [23]:
air_df

,PM1,PM10,PM25,latitude,longitude,timestamp
0,15.40,30.75,23.06,19.940984,50.062006,2022-12-20 00:00:00
1,15.30,38.80,24.29,19.942561,50.064539,2022-12-20 00:00:00
2,17.21,47.44,32.20,19.933919,50.059085,2022-12-20 00:00:00
3,15.68,40.76,28.61,19.946008,50.057447,2022-12-20 00:00:00
4,11.76,26.28,20.28,19.928368,50.062418,2022-12-20 00:00:00
...,...,...,...,...,...,...
43,20.87,34.98,50.06,19.953954,50.099239,2022-12-21 23:00:00
44,17.24,31.08,45.15,20.001157,50.065971,2022-12-21 23:00:00
45,15.93,27.25,37.69,19.905241,50.094220,2022-12-21 23:00:00
46,18.40,32.55,49.42,20.002049,50.054622,2022-12-21 23:00:00


### Create air pollution change visualisation with Folium

In [24]:
pm1_data = []
for time in air_df['timestamp'].unique():
    temp = []
    for index, record in air_df[air_df['timestamp'] == time].iterrows():
        temp.append([record['longitude'], record['latitude'], record['PM1']])
    pm1_data.append(temp)

## Create map

In [25]:
# create normal map
krakow_coords = [50.0496863, 19.944544]
map_krakow = folium.Map(krakow_coords, zoom_start=13)

In [26]:
# create heatmap
hm = plugins.HeatMapWithTime(
    pm1_data,
    display_index=True,
    index=air_df['timestamp'].apply(lambda timestamp: str(timestamp)).unique().tolist(),
    auto_play=True,
    radius=30,
    use_local_extrema=True,
)
# add heatmap on map 
hm.add_to(map_krakow)

map_krakow

In [17]:
map_krakow.save('pm1_vis01.html')